In [169]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [170]:
FILENAME = "/Volumes/RCONCI/2 digits colours/"

In [171]:
fulldatasize = int(images_dict["images"].shape[0]/2)
training_size = int(fulldatasize*0.99)
test_size = int(fulldatasize - training_size)

#### transform bmp with torch and then parse into training and test set (slow)

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
root = "/Volumes/RCONCI/2 digits fixed offset"
image_size = 64
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(),
    transforms.ToTensor(),])
train_kwargs = {'root':root, 'transform':transform}
dset = ImageFolder
train_data = dset(**train_kwargs)

x_train = np.ndarray((training_size, 64, 64))
for i in range(training_size):
    x_train[i,:,:] = train_data.__getitem__(i)[0][0,:,:].numpy()
    if i%1000 ==0:
        print(i)

        y_train = np.ndarray((training_size, 64, 64))
for i in range(10000+training_size):
    y_train[i,:,:] = train_data.__getitem__(i)[0][0,:,:].numpy()
    if i%1000 ==0:
        print(i)

#### transform with scipy and parse intro training and test set (bug)

In [114]:
import os
import imageio
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

root = "/Volumes/RCONCI/2 digits fixed offset"
x_train = np.ndarray((training_size, 64, 64))

#for i in range(10):
#    path = os.path.join('imageio:'+ root + '/test/orig/orig_{}.bmp'.format(i) )
#    images = imageio.imread(path, as_gray=1)
#    x_train[i,:,:] = resize(image, (64, 64), anti_aliasing=True)
    

#### import already transformed images with pickle and convert to training and test (fast)

In [205]:
images_dict = pickle.load(open( "{}images_dict.p".format(FILENAME), "rb" ) )
x_train = images_dict["images"][np.arange(0, 2*training_size, step=2),:,:,0]
y_train = images_dict["images"][np.arange(1, 2*training_size, step=2),:,:,0]
x_test = images_dict["images"][np.arange(2*training_size,2*fulldatasize, step=2),:,:,0]
y_test = images_dict["images"][np.arange(2*training_size+1, 2*fulldatasize, step=2),:,:,0]

x_train = x_train/x_train[0].max()
y_train = y_train/y_train[0].max()
x_test = x_test/x_test[0].max()
y_test = y_test/y_test[0].max()

x_train = np.expand_dims(x_train,3)
y_train = np.expand_dims(y_train,3)
x_test = np.expand_dims(x_test,3)
y_test =np.expand_dims(y_test,3)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(9900, 60, 60, 1)
(9900, 60, 60, 1)
(100, 60, 60, 1)
(100, 60, 60, 1)


### Deep autoencoder

In [206]:
x_train = x_train.reshape(-1, 60*60)
y_train = y_train.reshape(-1, 60*60)
x_test = x_test.reshape(-1, 60*60)
y_test = y_test.reshape(-1, 60*60)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(9900, 3600)
(9900, 3600)
(100, 3600)
(100, 3600)


In [211]:
2**

128

In [ ]:
z_dim=20

input_img = Input(shape=(3600,))
encoded = Dense(2**11, activation='relu')(input_img)
encoded = Dense(2**10, activation='relu')(encoded)
encoded = Dense(2**9, activation='relu')(encoded)
encoded = Dense(2**8, activation='relu')(encoded)
encoded = Dense(z_dim, activation='relu')(encoded)

decoded = Dense(2**8, activation='relu')(encoded)
decoded = Dense(2**9, activation='relu')(decoded)
decoded = Dense(2**10, activation='relu')(decoded)
decoded = Dense(2**11, activation='relu')(decoded)
decoded = Dense(3600, activation='sigmoid')(decoded)


In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

### convolutional autoencoder

In [191]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Reshape, Conv2DTranspose
from keras.models import Model
from keras import backend as K

z_dims = 10
    
input_img = Input(shape=(60, 60, 1))  # adapt this if using `channels_first` image data format
print(input_img.shape)
x = ZeroPadding2D(padding=(3, 3), data_format="channels_last")(input_img)
print(x.shape)
x = Conv2D(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32, 32, 32
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2D(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32, 16, 16
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2D(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32,  8,  8
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2D(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32,  4,  4
print(x.shape)
x = Reshape((-1, 32*4*4))(x) ## B, 512
print(x.shape)
x = Dense(2**8, activation='relu')(x) ## 256
print(x.shape)
encoded = Dense(z_dims*2, activation='relu')(x) ##2*z_dims
print(encoded.shape)

# at this point the representation is (z_dims *2)

x = Dense(2**8, activation='relu')(encoded) ## 256
print(x.shape)
x = Dense(512, activation='relu')(x) ## 512
print(x.shape)
x = Reshape((4, 4, 32))(x)   # B,  32,  4,  4
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2DTranspose(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32,  8,  8
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2DTranspose(32, kernel_size=4, strides=2, activation='relu', padding=('valid'), )(x) # B,  32, 16, 16
print(x.shape)
x = ZeroPadding2D(padding=(1, 1), data_format="channels_last")(x)
x = Conv2DTranspose(32, kernel_size=4, strides=2, activation='relu', padding=('valid'))(x) # B,  32, 32, 32
print(x.shape)
x = ZeroPadding2D(padding=(3, 3), data_format="channels_last")(x)
decoded = Conv2DTranspose(32, kernel_size=4, strides=2, activation='sigmoid', padding=('valid'))(x) # B,  nc, 64, 64
print(decoded.shape)


autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

(?, 60, 60, 1)
(?, 66, 66, 1)
(?, 32, 32, 32)
(?, 16, 16, 32)
(?, 8, 8, 32)
(?, 4, 4, 32)
(?, ?, 512)
(?, 1, 256)
(?, 1, 20)
(?, 1, 256)
(?, 1, 512)
(?, 4, 4, 32)
(?, ?, ?, 32)
(?, ?, ?, 32)
(?, ?, ?, 32)
(?, ?, ?, 32)


In [192]:
autoencoder.fit(x_train, y_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, y_test))

ValueError: Error when checking target: expected conv2d_transpose_41 to have shape (162, 162, 32) but got array with shape (60, 60, 1)

In [199]:
38*2+4-2

78

In [200]:
78*2+4-2

158